In [0]:
import pandas as pd
import numpy as np

from sklearn.metrics import f1_score
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.preprocessing import StandardScaler


from itertools import product
from sklearn import model_selection
from sklearn.model_selection import KFold

from tqdm.notebook import tqdm

import glob

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
!tar -xvzf "drive/My Drive/result.tar.gz"

Streaming output truncated to the last 5000 lines.
result/55.txt
result/550.txt
result/5500.txt
result/5501.txt
result/5502.txt
result/5503.txt
result/5504.txt
result/5505.txt
result/5506.txt
result/5507.txt
result/5508.txt
result/5509.txt
result/551.txt
result/5510.txt
result/5511.txt
result/5512.txt
result/5513.txt
result/5514.txt
result/5515.txt
result/5516.txt
result/5517.txt
result/5518.txt
result/5519.txt
result/552.txt
result/5520.txt
result/5521.txt
result/5522.txt
result/5523.txt
result/5524.txt
result/5525.txt
result/5526.txt
result/5527.txt
result/5528.txt
result/5529.txt
result/553.txt
result/5530.txt
result/5531.txt
result/5532.txt
result/5533.txt
result/5534.txt
result/5535.txt
result/5536.txt
result/5537.txt
result/5538.txt
result/5539.txt
result/554.txt
result/5540.txt
result/5541.txt
result/5542.txt
result/5543.txt
result/5544.txt
result/5545.txt
result/5546.txt
result/5547.txt
result/5548.txt
result/5549.txt
result/555.txt
result/5550.txt
result/5551.txt
result/5552.t

In [0]:
way =  "drive/My Drive/Colab Notebooks"

In [0]:
all_files = glob.glob("result" + "/*.txt")

In [6]:
prefix_l = len ("/result")
docs_to_dict = {}
start = pd.DataFrame()
for f_name in tqdm(all_files):
    with open(f_name) as f:
        title =' '
        doc_id = int(f_name[prefix_l:-4])
        for num_line, line in enumerate(f):
            title = ' '.join(word for word in line.split() if len(word)>2)
        docs_to_dict[doc_id] = title
        
print(len(docs_to_dict))


28026


Функция делает словарь {группа: [ (док ид, тайтл, таргет) ]}

In [10]:
train_data = pd.read_csv(way+'/train_groups.csv')
traingroups_titledata = {}
for i in tqdm(range(len(train_data))):
    new_doc = train_data.iloc[i]
    doc_group = new_doc['group_id']
    doc_id = new_doc['doc_id']
    target = new_doc['target']
    title = docs_to_dict[doc_id]
    if doc_group not in traingroups_titledata:
        traingroups_titledata[doc_group] = []
    traingroups_titledata[doc_group].append((doc_id, title, target))

In [11]:
len(traingroups_titledata)

129

Получаем фичи - пересечения и отклонение от средней длины

In [0]:
def getTrain(params):
    y_train = []
    X_train = []
    for new_group in tqdm(traingroups_titledata):
        docs = traingroups_titledata[new_group]
        ln = 0
        for j in range(0, len(docs)):
            doc_id_j, title_j, target_j = docs[j]
            words_j = set(title_j.strip().split())
            ln += len(words_j)
        ln /= len(docs)
        for k, (doc_id, title, target_id) in enumerate(docs):
            y_train.append(target_id)
            all_dist = []  
            words = set(title.strip().split())
            for j in range(0, len(docs)):
                if k == j:
                    continue
                doc_id_j, title_j, target_j = docs[j]
                words_j = set(title_j.strip().split())
                all_dist.append(len(words.intersection(words_j)))
            all_dist.sort(reverse=True)
            all_dist = all_dist[0:params]
            all_dist.append(abs(len(words) - ln))
            X_train.append(all_dist)
            
    X_train = np.array(X_train)
    y_train = np.array(y_train)
    
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    
    return X_train, y_train

In [13]:
a, b = getTrain(25)
print(a)


[[-0.56678894 -0.58967964 -0.61020255 ... -0.63135962 -0.62938674
  -0.49773158]
 [-0.51051429 -0.52034349 -0.54297045 ... -0.56901959 -0.57241417
  -0.57202099]
 [-0.62981656 -0.65901578 -0.66302778 ... -0.66332886 -0.66357029
  -0.46334971]
 ...
 [ 1.1274533   1.33395389  1.37602594 ...  1.67841826  1.6934665
   1.13741725]
 [ 0.03572502  0.1001261   0.11206236 ...  0.39325466  0.41239747
  -0.18275635]
 [-0.39421334 -0.38256013 -0.37873202 ... -0.29728102 -0.3005736
   0.49076003]]


сохраняем фичи в файл

In [0]:
df=pd.DataFrame(data=a[0:,0:], 
                index=[i for i in range(a.shape[0])],
                columns=['f'+str(i) for i in range(a.shape[1])])
df ['target'] = b

In [17]:
df

,f0,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,f12,f13,f14,f15,f16,f17,f18,f19,f20,f21,f22,f23,f24,f25,target
0,-0.566789,-0.589680,-0.610203,-0.606029,-0.604782,-0.602268,-0.599013,-0.595933,-0.596133,-0.622124,-0.626820,-0.627503,-0.628299,-0.635232,-0.634167,-0.637510,-0.635761,-0.634948,-0.636714,-0.634179,-0.631452,-0.629897,-0.632013,-0.631360,-0.629387,-0.497732,0
1,-0.510514,-0.520343,-0.542970,-0.553366,-0.563432,-0.561703,-0.557273,-0.553168,-0.547611,-0.543440,-0.542602,-0.545351,-0.543262,-0.552359,-0.555025,-0.552531,-0.550301,-0.549199,-0.551895,-0.571576,-0.575328,-0.572777,-0.570740,-0.569020,-0.572414,-0.572021,0
2,-0.629817,-0.659016,-0.663028,-0.664768,-0.667337,-0.663663,-0.661059,-0.658347,-0.658856,-0.660861,-0.665214,-0.662891,-0.661799,-0.660225,-0.666361,-0.666293,-0.666583,-0.664961,-0.664500,-0.663990,-0.664824,-0.663860,-0.665006,-0.663329,-0.663570,-0.463350,0
3,-0.533774,-0.550567,-0.542970,-0.539188,-0.535866,-0.529909,-0.525686,-0.539298,-0.539327,-0.536177,-0.535171,-0.533976,-0.556146,-0.557621,-0.572463,-0.570349,-0.568514,-0.569207,-0.567981,-0.564123,-0.584429,-0.585127,-0.584880,-0.581807,-0.582181,-0.616226,0
4,-0.536776,-0.550567,-0.550654,-0.543239,-0.547528,-0.546354,-0.617063,-0.613270,-0.611518,-0.610019,-0.611958,-0.612336,-0.608972,-0.607607,-0.607339,-0.607356,-0.606341,-0.606365,-0.606004,-0.605859,-0.604149,-0.603653,-0.614731,-0.612178,-0.609853,-0.559742,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11685,0.453658,0.610369,0.645117,0.705477,0.681305,0.728703,0.755851,0.773708,0.795619,0.803874,0.839560,0.849968,0.872743,0.877526,0.878935,0.840020,0.868896,0.848516,0.881261,0.911518,0.938515,0.901539,0.924953,0.943126,0.951195,-0.157429,0
11686,0.281833,0.408583,0.444381,0.465456,0.486218,0.415147,0.438851,0.441989,0.468984,0.490348,0.506405,0.526416,0.551920,0.551297,0.574436,0.589196,0.601308,0.609847,0.599017,0.625333,0.627555,0.639095,0.656294,0.663395,0.674471,-0.576151,0
11687,1.127453,1.333954,1.376026,1.393132,1.452108,1.434754,1.486868,1.540013,1.447707,1.466030,1.460048,1.421239,1.448680,1.493153,1.477200,1.508884,1.556779,1.591678,1.605152,1.637414,1.642345,1.616312,1.657088,1.678418,1.693466,1.137417,0
11688,0.035725,0.100126,0.112062,0.146441,0.181926,0.207936,0.235791,0.247812,0.260694,0.274875,0.273567,0.253419,0.253000,0.256638,0.276645,0.297254,0.318309,0.326874,0.341634,0.361506,0.380304,0.399808,0.382921,0.393255,0.412397,-0.182756,0


In [0]:
df.to_csv("fich_19_coins_1_avg.csv")

In [20]:
 a = np.array([1, 2, 3])
 b = []
 b = a#list(a)
 b

array([1, 2, 3])

In [0]:

splits = 6
kf = KFold(n_splits=splits)

def max_barrier(clf, X, y):
    """
    Подбираем порог линейной модели, 
    по достижению которого относим объект 
    к классу 1
    """
    (max_score, max_barr) = (0, 0)
    for barr in np.arange (0, 1, 0.01):
            probe = list(zip(*clf.predict_proba(X)))[1]
            score = f1_score(y, probe > barr)
#             print(barr, score)
            if score > max_score:
#                 print("AAaaaaaaaaaaaaaaaaa")
#                 print(barr, score)
                max_score, max_barr = score, barr
    return max_score, max_barr

def get_best(penalty_list, c_list, l1_ratio_list, fiches_list):
    """
    Делаем кросс валидацию и, таким образом, 
    подбираем оптимальные параметры.
    """
    max_result = -1
    for i, (penalty, C, l1_ratio, fiches) in tqdm(list(enumerate(product(penalty_list, c_list, l1_ratio_list, fiches_list)))):
        result = 0.0
        #X_tran, y_tran = getTrain(fiches)
        X_tran = a
        y_tran = b
        for train_index, test_index in kf.split(X_tran):
            X_tr, X_tst = X_tran[train_index], X_tran[test_index]
            y_tr, y_tst = y_tran[train_index], y_tran[test_index]
            clf = LogisticRegression(C=C, penalty=penalty, class_weight='balanced', solver='saga', l1_ratio=l1_ratio, max_iter = 10000)
            clf.fit(X_tr, y_tr)
            
            score, barrier = max_barrier(clf, X_tst, y_tst)
#             print(clf.predict(X_tst))
#             print(penalty, C, l1_ratio, barrier)
#             probe = clf.predict_proba(X_tst))
            result += score
        
        result /= splits
        print(result, fiches)
        if result > max_result:
            max_result = result
            params = {'C': C, 'penalty': penalty, 'l1_ratio': l1_ratio, 'barrier':barrier, 'fiches': fiches}
            print(params)
    
    print('max_f1_score =', max_result, 'with', params)
    return params

In [0]:
l1_ratio_list = [0.1]#np.arange(0.1, 1.0, 0.01)
c_list = [2442]#np.logspace(10, 1, )
fiches_list = [20]#np.arange(3, 25, 1)

penalty_list = ['elasticnet']

print(c_list)
print(l1_ratio_list)
print(fiches_list)

log_params = get_best(penalty_list, c_list, l1_ratio_list, fiches_list)


[2442]
[0.1  0.11 0.12 0.13 0.14 0.15 0.16 0.17 0.18 0.19 0.2  0.21 0.22 0.23
 0.24 0.25 0.26 0.27 0.28 0.29 0.3  0.31 0.32 0.33 0.34 0.35 0.36 0.37
 0.38 0.39 0.4  0.41 0.42 0.43 0.44 0.45 0.46 0.47 0.48 0.49 0.5  0.51
 0.52 0.53 0.54 0.55 0.56 0.57 0.58 0.59 0.6  0.61 0.62 0.63 0.64 0.65
 0.66 0.67 0.68 0.69 0.7  0.71 0.72 0.73 0.74 0.75 0.76 0.77 0.78 0.79
 0.8  0.81 0.82 0.83 0.84 0.85 0.86 0.87 0.88 0.89 0.9  0.91 0.92 0.93
 0.94 0.95 0.96 0.97 0.98 0.99 1.   1.01 1.02 1.03 1.04 1.05 1.06 1.07
 1.08 1.09]
[20]


0.5525202491143065 20
{'C': 2442, 'penalty': 'elasticnet', 'l1_ratio': 0.1, 'barrier': 0.5700000000000001, 'fiches': 20}
0.5522372472024258 20
0.552031137041572 20
0.5523856952456108 20
0.5522695110359643 20
0.5523092182709975 20
0.5522408286759465 20
0.5525405533132686 20
{'C': 2442, 'penalty': 'elasticnet', 'l1_ratio': 0.16999999999999998, 'barrier': 0.5700000000000001, 'fiches': 20}
0.5523856952456108 20
0.5523856952456108 20
0.5522372472024258 20
0.5523856952456108 20
0.55245190341766 20
0.5523856952456108 20
0.5524650200561857 20
0.5523275571454551 20
0.5524469388368004 20
0.552179585084757 20
0.5520480000008422 20
0.5523856952456108 20
0.5523856952456108 20
0.55245190341766 20
0.5523856952456108 20
0.5523856952456108 20
0.5523856952456108 20
0.5524083342260347 20
0.5523856952456108 20
0.552031137041572 20
0.5523856952456108 20
0.5522984907936155 20
0.5523856952456108 20
0.552031137041572 20
0.5523856952456108 20
0.5523856952456108 20
0.5523856952456108 20
0.5523856952456108 20
0.

ValueError: ignored

In [0]:
# X_train, y_train = getTrain(9)

тест надо чуть поменять еще одна фича же

In [0]:
test_data = pd.read_csv(way2+'/test_groups.csv')
testgroups_titledata = {}
for i in tqdm(range(len(test_data))):
    new_doc = test_data.iloc[i]
    doc_group = new_doc['group_id']
    doc_id = new_doc['doc_id']
    pair_id = new_doc['pair_id']
    title = docs_to_dict[doc_id]
    if doc_group not in testgroups_titledata:
        testgroups_titledata[doc_group] = []
    testgroups_titledata[doc_group].append((doc_id, pair_id, title))
    
X_test = []
pairs_id = []
for new_group in tqdm(testgroups_titledata):
    docs = testgroups_titledata[new_group]        
    ln = 0
    for j in range(0, len(docs)):
        *_, title_j = docs[j]
        words_j = set(title_j.strip().split())
        ln += len(words_j)
    ln /= len(docs)
    for k, (doc_id, pair_id, title) in enumerate(docs):
        all_dist = []
        words = set(title.strip().split())
        for j in range(0, len(docs)):
            if k == j:
                continue
            *_, title_j = docs[j]
            words_j = set(title_j.strip().split())
            all_dist.append(len(words.intersection(words_j)))
        
        all_dist.sort(reverse=True)
        all_dist = all_dist[0:20]#all_dist[0:log_params['fiches']]
        all_dist.append(abs(len(words) - ln))
        X_test.append(all_dist)
        pairs_id.append(pair_id)
X_test = np.array(X_test)
# X_test = scaler.transform(X_test)
print(X_test.shape)


(16627, 21)


In [0]:
scaler = StandardScaler()
X_test = scaler.fit_transform(X_test)

In [0]:
df=pd.DataFrame(data=X_test[0:,0:], 
                index=[i for i in range(X_test.shape[0])],
                columns=['f'+str(i) for i in range(X_test.shape[1])])
# df ['target'] = b

In [0]:
df

,f0,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,f12,f13,f14,f15,f16,f17,f18,f19,f20
0,-0.659096,-0.670561,-0.671301,-0.677275,-0.693355,-0.690898,-0.689171,-0.697756,-0.695778,-0.694168,-0.694754,-0.693079,-0.694578,-0.694276,-0.693190,-0.695121,-0.697131,-0.696816,-0.695210,-0.696421,-0.510075
1,-0.626125,-0.630312,-0.627589,-0.623952,-0.625164,-0.625620,-0.621028,-0.617285,-0.616641,-0.616564,-0.613087,-0.618814,-0.616187,-0.620200,-0.622339,-0.620243,-0.622012,-0.618846,-0.615725,-0.612686,-0.593345
2,-0.568243,-0.564069,-0.563359,-0.565016,-0.557947,-0.555322,-0.552884,-0.549520,-0.547261,-0.551154,-0.547299,-0.543389,-0.538983,-0.552195,-0.550245,-0.561863,-0.559844,-0.555412,-0.552406,-0.549541,-0.673663
3,-0.691334,-0.707455,-0.710553,-0.713759,-0.715761,-0.713996,-0.712918,-0.711521,-0.709870,-0.708580,-0.708366,-0.708164,-0.708831,-0.707635,-0.706863,-0.706543,-0.707493,-0.706067,-0.707335,-0.707403,-0.495901
4,-0.409252,-0.407267,-0.417058,-0.441532,-0.437151,-0.441840,-0.453766,-0.447873,-0.438855,-0.431422,-0.429335,-0.426189,-0.419020,-0.413757,-0.412272,-0.437489,-0.431623,-0.453654,-0.447324,-0.456197,-0.403387
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16622,-0.629789,-0.639536,-0.644539,-0.652017,-0.650492,-0.645706,-0.650970,-0.658580,-0.654583,-0.650931,-0.648249,-0.648985,-0.651819,-0.651773,-0.653414,-0.654509,-0.653096,-0.651884,-0.650753,-0.653867,-0.546724
16623,-0.555055,-0.559877,-0.560683,-0.554726,-0.576456,-0.571390,-0.574566,-0.585520,-0.581951,-0.577761,-0.577924,-0.582842,-0.579367,-0.578911,-0.577591,-0.573285,-0.568910,-0.564663,-0.564531,-0.572878,-0.614049
16624,-0.507431,-0.522144,-0.545517,-0.539758,-0.531644,-0.523185,-0.515715,-0.509285,-0.502815,-0.497940,-0.506465,-0.509738,-0.518791,-0.519407,-0.521656,-0.521251,-0.527465,-0.522374,-0.526809,-0.522087,-0.627179
16625,-0.482520,-0.503696,-0.553546,-0.553791,-0.555998,-0.556326,-0.554949,-0.563285,-0.558102,-0.552263,-0.547299,-0.542229,-0.542546,-0.537623,-0.534086,-0.536480,-0.535236,-0.530303,-0.533545,-0.537187,-0.684327


In [0]:
# df.iloc[2222]
df.to_csv("fich_20_coins_1_avg_test.csv")

In [0]:
# log_params= {'C': 100, 'penalty': 'elasticnet', 'l1_ratio': 0.1, 'barrier':0.26, 'fiches': 9}

In [0]:
log_clf = LogisticRegression(C=100, penalty='elasticnet', class_weight='balanced', solver='saga', l1_ratio=0.1, max_iter = 10000)
log_clf.fit(a, b)

LogisticRegression(C=100, class_weight='balanced', dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=0.1,
                   max_iter=10000, multi_class='auto', n_jobs=None,
                   penalty='elasticnet', random_state=None, solver='saga',
                   tol=0.0001, verbose=0, warm_start=False)

In [0]:
X_test = scaler.fit_transform(X_test)

In [0]:
X_test.shape

(16627, 21)

In [0]:
# scaler = StandardScaler()
# X_test = scaler.fit_transform(X_test)
probes = list(zip(*log_clf.predict_proba(X_test)))[1]

probes = list(probes)
# predict = np.asarray(probes > log_params['barrier'], dtype=int)
pred = []
for i in range(len(probes)):
    pred.append(int(probes[i] > log_params['barrier']))
  
predict = np.array(pred)
print(predict[5])

1


In [0]:
test_data["target"] = pd.Series(predict)
with open("predict_20.csv", "w") as f:
    f.write(test_data.to_csv(columns=("pair_id", "target"), index=False))
print("good")

good
